#### Thoughts...
* Есть мультистраничные excel'и
* Одна колонка таблицы может быть объединенной
* У одной записи могут быть подзаписи с дробным номером
* В этой версии подразумевается структура по СН-2012
* В некоторых сметах есть несколько локальных подсмет

In [1]:
import pandas as pd
import string
import os
from glob import glob
import random
import json
import regex as re
from IPython.display import JSON,display,display_json
from tqdm import tqdm

In [2]:
class WrongSmeta(Exception):
    pass

In [49]:
path = "./soure_data/smeth_conc/smety_ishod/"
#path = "./soure_data/СН-ТСН/СН-2012/"
!find $path > files.txt
with open("files.txt") as f:
    files = f.readlines()
    files = list(map(lambda x: "".join(x.split('\n')), files))
allowed_ext = ('xls', 'xlsx')
excels = list(filter(lambda x: x.lower().endswith(allowed_ext), files))

In [81]:
#heet = "soure_data/smeth_conc/smety_ishod/1771608138821000208/Проектно-сметая документация (1).xlsx"
sheet = "soure_data/smeth_conc/smety_ishod/87702132064210000340000/Прил3 к ТЗ сметная докмуентация Благоустройство сквера ЦАК.xlsx"
#heet = "soure_data/smeth_conc/smety_ishod/2770182732021001448/ГБУ ТЦСО _Марьино_ филиал _Люблино_1 (1).xlsx"
#sheet = './soure_data/smeth_conc/smety_ishod/2774394681021000083/ЛС 3 (1).xls'
#sheet = './weird.xlsx'
#sheet = "soure_data/smeth_conc/smety_ishod///2770182732021001448/ГБУ ТЦСО _Марьино_ филиал _Люблино_1 (1).xlsx"
#sheet = random.choice(excels)
sheet = "./soure_data/smeth_conc/smety_ishod/2772494153421000012/Смета (2).xls"

In [82]:
sheet

'./soure_data/smeth_conc/smety_ishod/2772494153421000012/Смета (2).xls'

In [83]:
excel_file = pd.ExcelFile(sheet)
print("Все книги:",excel_file.sheet_names)
for i in excel_file.sheet_names:
    if("смета" in i.lower()):
        choosen_name = i
        print(f"Выбрана книга: {i}")
        break
#df = pd.read_excel(sheet,header=None)
df = excel_file.parse(sheet_name=choosen_name,header=None)
df = df.dropna(how="all").dropna(axis=1,how="all")
df = df.astype(str)
df = df.reset_index(drop=True)
df.columns = range(df.columns.size)

Все книги: ['Смета СН-2012 по гл. 1-5', 'Дефектная ведомость', 'RV_DATA', 'Расчет стоимости ресурсов', 'Source', 'SourceObSm', 'SmtRes', 'EtalonRes']
Выбрана книга: Смета СН-2012 по гл. 1-5


In [84]:
#Определение начала таблицы
def series_startswith(ser, pattern):
    index = 0
    for val in ser:
        if(val == pattern[index]):
            index+=1
        elif(val == 'nan'):
            continue
        else:
            return False
        if(index == len(pattern)):
            return True
    return False

one2elvn = [str(_) for _ in range(1,12)]
for index, row in df.iterrows():
    if(series_startswith(row, one2elvn)):
        print(index)
        table_title = index
        break

21


In [85]:
# Маппинг номеров колонок таблицы в случае объединенных excel ячеек
column_map = {}
index = 1
for ind, val in enumerate(df.iloc[table_title]):
    if(val == str(index)):
        column_map[index] = ind
        index+=1
        
columns_mask = list(column_map.values())
column_map


{1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 8: 7, 9: 8, 10: 9, 11: 10}

In [86]:
# Проверка соответствия колонок по СН-2012
ser = df.iloc[table_title-1][columns_mask].apply(lambda x: "" if x == "nan" else x)[::]
for index, row in df[:table_title-1][::-1].iterrows():
    ser += row[columns_mask].apply(lambda x: "" if x == "nan" else x)
    if("№" in ser[0]):
        break

def contains(pattern: str or tuple, s: str, lower = True, to_delete = ['-','\n']):
    if(to_delete):
        s = re.sub('|'.join(to_delete),"", s)
    if(lower):
        pattern = tuple(map(lambda x:x.lower(), pattern))
        s = s.lower()
    if(type(pattern) is str):
        return pattern in s
    flag = True
    for val in pattern:
        flag *= val in s
    return flag
        
    pass
if(contains("№", ser[column_map[1]]) and  
   contains("шифр", ser[column_map[2]]) and 
   contains("наименовани", ser[column_map[3]]) and 
   contains(("ед","изм"), ser[column_map[4]]) and
   contains(("кол","во","ед"), ser[column_map[5]]) and
   contains(("цена", "ед"), ser[column_map[6]]) and 
   contains(("коэф", "поправоч"), ser[column_map[7]]) and 
   contains(("коэф", "зимн"), ser[column_map[8]]) and 
   contains(("коэф", "пересч"), ser[column_map[9]]) and 
   contains("всего", ser[column_map[10]]) and 
   contains(("справ", "зтр","ед"), ser[column_map[11]]) and
   (contains("стоим", ser[column_map[11]]) or contains(("ст", "ть"), ser[column_map[11]]))):
    print("Смета подходит под СН-2012")
else:
    raise WrongSmeta("Формат сметы не подходит")

Смета подходит под СН-2012


In [87]:
ser

0                                                №№ п/п
1                         Шифр расценки и коды ресурсов
2                           Наименование работ и затрат
3                                     Единица измерения
4                                         Кол-во единиц
5                                 Цена на ед. изм. руб.
6                                   Попра-вочные коэфф.
7                             Коэфф. зимних удоро-жаний
8                                      Коэфф. пересчета
9                                    ВСЕГО затрат, руб.
10    Ст-ть ед. с начислен.ЗТР, всего чел.-часСправочно
Name: 20, dtype: object

In [88]:
# Определение разделов и подразделов
current_section = None
current_subsection = None
current_item = 0
item_indices = {0:[None,None,None,None]} # Началный индекс, Конечный индекс, раздел, подраздел
#Границы записей явлеяются отрезком [], учитывать при использовании slice

for index, row in tqdm(df[table_title+1:].iterrows()):
    # Определения разделов и подразделов
    if(row.str.lower().str.contains("подраздел").any()):
        if((row.str.lower().str.contains("подраздел") * row.str.lower().str.contains("итог")).any()):
            #print(f"    Конец {current_subsection}")
            current_subsection = None
            item_indices[current_item][1] = index-1
        else:
            # Иногда в одной линии два раза встречается подраздел
            for cell in row:
                if(cell!="nan" and "подраздел" in cell.lower()):
                    current_subsection = cell
                    break
                    
            #print(f"    Начало {current_subsection}")
    elif(row.str.lower().str.contains("раздел").any()):
        if((row.str.lower().str.contains("раздел") * row.str.lower().str.contains("итог")).any()):
            #print(f"Конец {current_section}")
            current_section = None
            item_indices[current_item][1] = index-1
        else:
            # Иногда в одной линии два раза встречается раздел
            for cell in row:
                if(cell!="nan" and "раздел" in cell.lower()):
                    current_section = cell
                    break
            
            #print(f"Начало {current_section}")
    elif(row.str.lower().str.contains("смет").any() * \
         row.str.lower().str.contains("итог").any()):
        if(item_indices[current_item][1] is None):
            item_indices[current_item][1] = index-1
         
    # Определение номера и границ записи
    if( row[column_map[1]].isnumeric() and row[column_map[1]] == str(current_item+1)):
        if(item_indices[current_item][1] is None):
            item_indices[current_item][1] = index-1
        current_item += 1
        #print(f"****Запись**** At index {index} found new item num. {current_item}")
        item_indices[current_item] = [index,None,current_section,current_subsection]

#0 запись исскуственная
del item_indices[0]

assert item_indices, "No items were found. Cannot continue"

1496it [00:01, 800.82it/s]


In [95]:
def check_change_type(col_type, value):
    if(type(value) is col_type):
        return value
    if(col_type is float):
        value = re.sub("[^0-9|\.|,]","",value)
        try:
            return float(value)
        except Exception as e:
            if("," in value and "." not in value):
                try:
                    value = re.sub(",",".",value)
                    return float(value)
                except Exception as e:
                    print(value)
                    raise ValueError()

column_names = {1:"Number",
                2:"Code",
                3:"Name",
                4:"Unit",
                5:"Amount",
                6:"Price_per_unit",
                7:"Correction_coof",
                8:"Winter_coof",
                9:"Recalc_coof",
                10:"Total",
                11:"Additional"}

column_types = {1:float,
                 2:str,
                 3:str,
                 4:str,
                 5:float,
                 6:float,
                 7:float,
                 8:float,
                 9:float,
                 10:float,
                 11:float}

columns_mask = list(column_map.values())
value_mask = list(map(lambda x: x[1],filter(lambda x: x[0]>3, column_map.items())))
    
all_items = []

for item_to_parse in range(1, len(item_indices)+1):
    zap = df.iloc[item_indices[item_to_parse][0]:item_indices[item_to_parse][1]+1].reset_index(drop=True)
    #Парсинг отдельной записи
    main_work_dict = {}
    for col in range(1, 12):
        if(zap.iloc[0][column_map[col]] != "nan"):
            main_work_dict[column_names[col]] = check_change_type(column_types[col],zap.iloc[0][column_map[col]])

    main_work_dict["Sub_works"] = []
    main_work_dict["Related_works"] = []

    for index, row in zap[1:].iterrows():
        if(len(set(row[value_mask])) == 1 and row[value_mask[0]] == "nan"):
            continue
        if(row[column_map[3]] == 'nan'):
            continue
        if(row.str.lower().str.contains("итого").any()):
            continue
        
        if(row[column_map[1]] != 'nan' or row[column_map[2]] != 'nan'):
            related_work_dict = {}
            for col in range(1, 12):
                if(row[column_map[col]] != "nan"):
                    related_work_dict[column_names[col]] = check_change_type(column_types[col],row[column_map[col]])
            main_work_dict["Related_works"].append(related_work_dict)
        else:
            sub_work_dict = {}
            for col in range(1, 12):
                if(row[column_map[col]] != "nan"):
                    sub_work_dict[column_names[col]] = check_change_type(column_types[col],row[column_map[col]])
            main_work_dict["Sub_works"].append(sub_work_dict)
        
    for index, row in zap[::-1].iterrows():
        first = None
        second = None
        for val in row[::-1]:
            if(val != 'nan'):
                if(second):
                    first = val
                    break
                else:
                    second = val
        if(first and second and re.search('[a-zA-Zа-яА-Я]', first+second) is None):
            main_work_dict["Total"] = check_change_type(float,first)
            main_work_dict["Unit_cost_with_additions"] = check_change_type(float,second)
            break
            
    main_work_dict["Section"] = item_indices[item_to_parse][2] 
    main_work_dict["Sub_section"] = item_indices[item_to_parse][3]
    
    if("Code" not in main_work_dict.keys()):
        main_work_dict["Code"] = ""
    
    all_items.append(main_work_dict)

In [96]:
sheet



'./soure_data/smeth_conc/smety_ishod/2772494153421000012/Смета (2).xls'

In [97]:
all_items[0]

{'Number': 1.0,
 'Name': 'Площадь Ремонта АБП',
 'Amount': 195.0,
 'Sub_works': [],
 'Related_works': [],
 'Total': 0.0,
 'Unit_cost_with_additions': 0.0,
 'Section': 'Раздел: Ремонт асфальтобетонного покрытия проездов  на дворовых территория (195м2)',
 'Sub_section': None,
 'Code': 'empty'}

In [98]:
if("chaps" not in vars().keys()):
    chaps = {}
    for i in range(1,8):
        with open(f"./SN_2012_CHAPTER_{i}.json", "r") as f:
            chaps[i] = json.load(f)


In [99]:

#display(JSON(item_in_q))
mapping = []

for ind, item_in_q  in enumerate(tqdm(all_items)):
    all_items[ind]["Source_from_reference"] = None

    found = False
    for key,chap in chaps.items():
        for z in chap:
            if(item_in_q["Name"].lower() in z["Name"].lower() \
               or item_in_q["Code"].lower() in z["Code"].lower()):
                all_items[ind]["Source_from_reference"] = z
                all_items[ind]["Source_from_reference"]["From_chapter"] = key
                found = True
                break
        if(found):
            break
    if(not found):
        print(f'---> Не найден образец для записи номер {item_in_q["Number"]} с кодом: {item_in_q["Code"]} \n Наименование: {item_in_q["Name"]}')
    

  2%|███▏                                                                                                                                            | 4/183 [00:00<00:12, 14.87it/s]

---> Не найден образец для записи номер 1.0 с кодом: empty 
 Наименование: Площадь Ремонта АБП
---> Не найден образец для записи номер 3.0 с кодом: empty 
 Наименование: Масса мусора (195м2*0,06м*2,4т/м3=28,08тн)


  7%|█████████▍                                                                                                                                     | 12/183 [00:00<00:12, 14.23it/s]

---> Не найден образец для записи номер 10.0 с кодом: empty 
 Наименование: Площадь Ремонт тротуара
---> Не найден образец для записи номер 12.0 с кодом: empty 
 Наименование: Масса мусора  (1293м2*0,05м*2,4т/м3=155,16тн)


 12%|█████████████████▏                                                                                                                             | 22/183 [00:01<00:11, 14.53it/s]

...

 93%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉          | 170/183 [00:11<00:00, 24.48it/s]

---> Не найден образец для записи номер 167.0 с кодом: empty 
 Наименование: Площадь кровли 8,6мх6м=51,6м2х10шт=516м2
---> Не найден образец для записи номер 170.0 с кодом: 1.5-3104-1-1/1
Поправка: СН-2012.1-1. Гл.1 Сб.5 п.3. 8.1 
 Наименование: Разборка кладки стен из кирпича простых (1,8м х7,8м х0,4мх5 стен)(общая стена)+ боковая (5,3мх1м*0,4мх9шт)=471,6м3
Поправка: СН-2012.1-1. Гл.1 Сб.5 п.3. 8.1  Наименование: При разборке без сохранения материалов кладки стен из кирпича


 96%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊      | 175/183 [00:11<00:00, 29.51it/s]

---> Не найден образец для записи номер 172.0 с кодом: empty 
 Наименование: Площадь  полов (5,3м*7,8м*10веранд=413,4м2
---> Не найден образец для записи номер 176.0 с кодом: 5.3-3203-7-8/1
Поправка: СН-2012 О.П. п.22 
 Наименование: ДЕМОНТАЖ, Установка дополнительных бетонных столбов
---> Не найден образец для записи номер 177.0 с кодом: empty 
 Наименование: масса мусора


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 183/183 [00:12<00:00, 15.14it/s]


In [100]:
with open("all_items.json", "w") as f:
    json.dump(all_items, f)
          

In [80]:
f"from_{sheet.split('/')[-2]}.json"

'from_87702132064210000340000.json'